In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

import Model
import utils
import config

# Reload in case of edits
importlib.reload(Model)


class TremorModelTrainer:
    def __init__(self, config):
        self.default_path = config.default_path
        self.classes = config.classes
        self.dataset_info = config.dataset
        self.info_labels = config.Info
        self.results = []
        self.model = None


    def train(self, path, train_ratio=0.8):
        feature_set, labels = utils.get_dataset(path, self.classes, show_labels=False)
        X_train, y_train, X_test, y_test = utils.split_data(feature_set, labels, ratio=train_ratio)

        self.model = Model.Original_model_V1(X_train.shape[1:])

        history, self.model = Model.Train_model(
            self.model, X_train, y_train, X_test, y_test,
            set_epoch=50, set_batch_size=256, Model_name='V0',
            set_verbose=False, save_model_set=True
        )

        acc = self.model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Accuracy of test dataset using model V0: {acc * 100:.4f}%")
        return float(np.max(history.history['val_accuracy']) * 100)


    def train_multiple_dataset(self, path, X_train, y_train, X_test, y_test):
        self.model = Model.Original_model_V1(X_train.shape[1:])

        history, self.model = Model.Train_model(
            self.model, X_train, y_train, X_test, y_test,
            set_epoch=50, set_batch_size=256, Model_name='V0',
            set_verbose=False, save_model_set=True
        )

        acc = self.model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Accuracy of test dataset using model V0: {acc * 100:.4f}%")
        return float(np.max(history.history['val_accuracy']) * 100)


    def test(self, path):
        feature_set, labels = utils.get_dataset(path, self.classes, show_labels=False)
        _, _, X_test, y_test = utils.split_data(feature_set, labels, ratio=0)
        acc = self.model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Accuracy on unseen dataset: {acc * 100:.4f}%")
        return float(np.round(acc * 100, 2))


    def run_all_sessions_training_K_data(self, K):
        X_train_all, y_train_all = [], []
        X_test_all, y_test_all = [], []

        for idx, session_info in enumerate(self.dataset_info):
            print(f"\n{'='*40}\nDataset {idx + 1}/{len(self.dataset_info)} - Session {session_info}\n{'='*40}")
            path = os.path.join(self.default_path, f'{session_info}/raw/')

            if idx < K:
                feature_set, labels = utils.get_dataset(path, self.classes, show_labels=False)
                X_train, y_train, X_test, y_test = utils.split_data(feature_set, labels, ratio=0.99)

                # Stack cumulatively
                X_train_all.append(X_train)
                y_train_all.append(y_train)
                X_test_all.append(X_test)
                y_test_all.append(y_test)

                # Concatenate all so far
                X_train_stacked = np.concatenate(X_train_all, axis=0)
                y_train_stacked = np.concatenate(y_train_all, axis=0)
                X_test_stacked = np.concatenate(X_test_all, axis=0)
                y_test_stacked = np.concatenate(y_test_all, axis=0)
                acc = 0
                print(X_train_stacked.shape, y_train_stacked.shape), X_test_stacked.shape, y_test_stacked.shape

            elif idx == K:
                acc = self.train_multiple_dataset(path, X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked)

            else:
                acc = self.test(path)

            self.results.append(acc)


    def run_all_sessions_training_one_data(self):
        for idx, session_info in enumerate(self.dataset_info):
            print(f"\n{'='*40}\nDataset {idx + 1}/{len(self.dataset_info)} - Session {session_info}\n{'='*40}")
            path = os.path.join(self.default_path, f'{session_info}/raw/')

            if idx == 0:
                acc = self.train(path)
            else:
                acc = self.test(path)

            self.results.append(acc)


    def save_results(self, filepath):
        df = pd.DataFrame({
            'Info': self.info_labels,
            'Info_Set': self.dataset_info,
            'Accuracy': self.results
        })
        df.to_csv(filepath, index=False)
        print(f"Results saved to {filepath}")
        return df


In [2]:
if __name__ == "__main__":
    for K in [1, 10, 14, 18]:
        trainer = TremorModelTrainer(config)
        trainer.run_all_sessions_training_K_data(K)
        df = trainer.save_results(f'../Results/Baseline_results_train_with_{K}data.csv')


Dataset 1/26 - Session Exp_2025-05-27/E8331D05289A/
(8539, 4, 14, 1) (8539,)

Dataset 2/26 - Session Exp_2025-06-18/E9AD0E7DCC2B/
Start Training (total epochs: 50)...
Finish Training! (Model is saved)
Maximum training accuracy : 90.22%
Maximum validation accuracy : 94.19%
Accuracy of test dataset using model V0: 91.8605%

Dataset 3/26 - Session Exp_2025-06-20-v1/E9AD0E7DCC2B/
Accuracy on unseen dataset: 80.6125%

Dataset 4/26 - Session Exp_2025-06-20-v2/E9AD0E7DCC2B/
Accuracy on unseen dataset: 85.2084%

Dataset 5/26 - Session Exp_2025-06-20-v3/E9AD0E7DCC2B/
Accuracy on unseen dataset: 85.0267%

Dataset 6/26 - Session Exp_2025-06-20-v4/E9AD0E7DCC2B/
Accuracy on unseen dataset: 70.2815%

Dataset 7/26 - Session Exp_2025-06-20-v5/E9AD0E7DCC2B/
Accuracy on unseen dataset: 71.4319%

Dataset 8/26 - Session Exp_2025-06-20-v6/E9AD0E7DCC2B/
Accuracy on unseen dataset: 77.2206%

Dataset 9/26 - Session Exp_2025-06-20-v7/E9AD0E7DCC2B/
Accuracy on unseen dataset: 70.0256%

Dataset 10/26 - Session 